# Data retrival

In [2]:
%load_ext autoreload
%autoreload 2
import myfunctions as f
f.today()

'20180313'

In [3]:
# python version
import sys
print "system_info: %s"%sys.version
# current working directory
import os
print "path_info: %s"%os.getcwd() 

system_info: 2.7.14 |Anaconda custom (64-bit)| (default, Oct 27 2017, 18:21:12) 
[GCC 7.2.0]
path_info: /home/yueliu/Desktop/workspace_yue/Documentation_201712


In [4]:
# setup elastic search
f.initialise_es(1)
foodpanda, deliveroo, wte, websites = f.delivery_para()

{
  "name" : "star7",
  "cluster_name" : "star",
  "cluster_uuid" : "VDcV69ThThyb00sd6PgeMA",
  "version" : {
    "number" : "2.4.5",
    "build_hash" : "c849dd13904f53e63e88efc33b2ceeda0b6a1276",
    "build_timestamp" : "2017-04-24T16:18:17Z",
    "build_snapshot" : false,
    "lucene_version" : "5.5.4"
  },
  "tagline" : "You Know, for Search"
}



# To retrive food data

In [5]:
doc_type = "food"

In [6]:
%%time
website = websites[0]
f.initialise_es(0)
website.update({"all_rec_"+doc_type : f.retrive_from_es(website, doc_type)})

CPU times: user 2h 32min 15s, sys: 10min 4s, total: 2h 42min 20s
Wall time: 2h 43min 3s


In [7]:
%%time
website = websites[1]
f.initialise_es(0)
website.update({"all_rec_"+doc_type : f.retrive_from_es(website, doc_type)})

CPU times: user 1h 31min 17s, sys: 5min 15s, total: 1h 36min 32s
Wall time: 1h 37min 16s


In [8]:
%%time
website = websites[2]
f.initialise_es(0)
website.update({"all_rec_"+doc_type : f.retrive_from_es(website, doc_type)})

CPU times: user 14min 50s, sys: 30.9 s, total: 15min 21s
Wall time: 15min 27s


## Data: keep all index and dates

In [9]:
### fomulate location
def fomulate_loc(w):
    """formulate storage location according to es data"""
    w['all_rec_food']["loc"] = w['all_rec_food']["_index"] + "/" + \
    w["food"] + "/" + \
    w['all_rec_food']["_id"]

def fomulate_timestamp(w):
    if w["_index"] == 'what_to_eat':
        w['all_rec_food']["timestamp"] = w['all_rec_food']['_source.startTimestampGMT']
    else:
        d = f.retrive_from_es(w, "crawling_cycle")
        cycle_time = d.set_index(w['cycle_id']).to_dict()[w['cycleStart']]
        # remove na for food names
        w['all_rec_food'] = w['all_rec_food'][~w['all_rec_food'][w["food_name"]].isnull()]
        # assign time according to food cycle
        w['all_rec_food']["timestamp"] = w['all_rec_food'][w['food_cycle']].apply(lambda s: cycle_time[s])

def formated(w):
    cols = [w['food_name'], w['vendor_name'], "timestamp", "loc"]
    temp = w['all_rec_food'][cols]
    temp.columns = ["food_name", "vendor_name","timestamp", "loc" ]
    temp["locs"] = temp["timestamp"]+ "****" + temp["loc"]
    temp["locs"] = temp["locs"].apply(lambda s: tuple(s.split("****")))
    temp = temp[["food_name", "vendor_name", "locs" ]]
    w["formated"] = temp

In [10]:
for w in websites:
    fomulate_loc(w)     
    fomulate_timestamp(w) 
    formated(w)
    print "%s:: number of records: %d"%(w["_index"],
                                        w['all_rec_food'].shape[0])

foodpanda:: number of records: 2032443
deliveroo:: number of records: 1459312
what_to_eat:: number of records: 226512


# To save food data

In [13]:
for w in websites:
    file_name = "__".join([w["_index"], "food", f.today()]) + ".p"
    f.save_file(w['all_rec_food'], file_name)

saved: foodpanda__food__20180314.p
saved: deliveroo__food__20180314.p
saved: what_to_eat__food__20180314.p


In [14]:
import pandas as pd
file_name = "__".join(["all_food_vendor", f.today()]) + ".p"
df1 = pd.concat([w["formated"] for w in websites]) 
df = df1.groupby(["food_name", "vendor_name"])["locs"].apply(list).to_frame().reset_index()

In [15]:
print "all:: number of records: %d"%(df.shape[0])
f.save_file(df, file_name)

all:: number of records: 491732
saved: all_food_vendor__20180314.p


# Clean the food items: get clean name

In [16]:
# drink_list = f.retrive_drinks()
df = f.retrive_file(file_name)
# df["clean_name"] = df["food_name"].apply(lambda s: f.clean_name_v4(s))
df["clean_name"] = df["food_name"].apply(lambda s: f.clean_name_v2(s))
# df1 = df[~df['clean_name'].isin(drink_list)]
# print ("Got %d non-drink/dessert food-vendor records" % df1.shape[0])
# print ("Got %d unique non-drink/dessert food " % 
#        df1["clean_name"].nunique())

file_name = "__".join(["all_food_vendor","clean", f.today()]) + ".p"
f.save_file(df, file_name)

number of drinks in drink list: 22643
retrived: all_food_vendor__20180314.p
saved: all_food_vendor__clean__20180314.p


In [17]:
df["lemma_name"] = df["clean_name"].apply(lambda s: f.lemma_name(s))

In [18]:
file_name = "__".join(["all_food_vendor","lemma", f.today()]) + ".p"
f.save_file(df, file_name)

saved: all_food_vendor__lemma__20180314.p


In [22]:
df["lemma_name"].nunique()

136163

In [23]:
df.sample(2, random_state=2008)

,food_name,vendor_name,locs,clean_name,lemma_name
362606,Preserved Radish Bean Curd,The One Place - Eunos,"[(2017-07-03T17:55:01.668428, deliveroo/food/4...",preserved radish bean curd,preserve radish bean curd
108552,Brownie (Slice),Island Creamery - Joo Chiat,"[(2017-07-03T17:55:01.668428, deliveroo/food/6...",brownie slice,brownie slice


In [26]:
food_names = sorted([s for s in set(df["lemma_name"].tolist()) if (s!="" and s==s)])
print "number of unique food items: %d"%len(food_names)

number of unique food items: 136162


In [28]:
file_name = "food_items_" + f.today() + ".csv"
import csv
with open(file_name, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in food_names:
        writer.writerow([val])  
print ("saved: %s" % file_name)

saved: food_items_20180314.csv
